In [1]:
import glob
import matplotlib.pyplot as plt
import matplotlib
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import sys
from IPython.display import display


In [34]:
basedir = os.path.join('Y:\\', 'Dissertation')
languages = set()
repl_languages = set()
addl_languages = set()

# Get frequencies
freq = os.path.join(basedir, "count_evals", "wiki_top_3_per_algo.csv")
freq_wiki_df = pd.read_csv(freq, header=0)
for index, stuff in freq_wiki_df.iterrows():
        languages.add(stuff['Language'])
        
freq = os.path.join(basedir, "count_evals", "subs_top_3_per_algo.csv")
freq_subs_df = pd.read_csv(freq, header=0)
for index, stuff in freq_subs_df.iterrows():
        languages.add(stuff['Language'])

# Get replication
repl = os.path.join('Z:\\', 'Dissertation', "scores", "top_3_by_language.csv")
repl_df = pd.read_csv(repl, header=0)
for index, stuff in repl_df.iterrows():
    languages.add(stuff['Language'])
    repl_languages.add(stuff['Language'])
                
# Finally the additional datasets
addl = os.path.join(basedir, "scores", "top_3_per_algo.csv")
addl_df = pd.read_csv(addl, header=0)
for index, stuff in addl_df.iterrows():
        languages.add(stuff['Language'])
        addl_languages.add(stuff['Language'])


In [39]:
rows = list()
pd.options.mode.copy_on_write = True
for lang in languages:
    row = dict()
    row["Language"] = lang
    df_lang = freq_wiki_df.loc[freq_wiki_df['Language'] == lang]
    if len(df_lang) > 0:
        df_lang.sort_values(by=['Score',"Dimensions","Window"], ascending=[False, True, True], inplace=True)
        dim = df_lang['Dimensions'].iloc[0]
        win = df_lang['Window'].iloc[0]
        row["Frequency: Wikipedia"] = f'{dim}-{win}'
    else:
        row["Frequency: Wikipedia"] = ''
     
    if len(df_lang) > 0:
        df_lang = freq_subs_df.loc[freq_subs_df['Language'] == lang]
        df_lang.sort_values(by=['Score',"Dimensions","Window"], ascending=[False, True, True], inplace=True)
        dim = df_lang['Dimensions'].iloc[0]
        win = df_lang['Window'].iloc[0]
        row["Frequency: Subtitles"] = f'{dim}-{win}'
    else:
        row["Frequency: Subtitles"] = ''
    
    if lang in repl_languages:
        filename = f'{lang}_scores.csv'
        repl = os.path.join('Z:\\', 'Dissertation', "scores", filename)
        repl_df = pd.read_csv(repl, header=0)
    else:
        repl_df = pd.DataFrame()
        
    if lang in addl_languages:
        filename = f'{lang}_scores.csv'
        addl = os.path.join(basedir, "scores", filename)
        addl_df = pd.read_csv(addl, header=0)
    else:
        addl_df = pd.DataFrame()
     
    if repl_df.empty and addl_df.empty:
        row['Imagineability'] = ''
        row['Age of Acquisition'] = ''
        row['Valence'] = ''
        df_lang = pd.DataFrame()
    elif repl_df.empty:
        df_lang = addl_df
    elif addl_df.empty:
        df_lang = repl_df
    else:
        # Concatenate replication and additional datasets
        df_lang = pd.concat([repl_df, addl_df])
    
    # See if we have data for this language in the norms set
    if not df_lang.empty:
        # Look for imagine
        df_imag = df_lang[df_lang.Norm.str.match('^imagine$|^imagine.*_M.*', case=False)]
        if len(df_imag) > 0:
            df_imag.sort_values(by=['Score',"Dimensions","Window"], ascending=[False, True, True], inplace=True)
            dim = df_imag['Dimensions'].iloc[0]
            win = df_imag['Window'].iloc[0]
            row['Imagineability'] = f'{dim}-{win}'
        else:
            row['Imagineability'] = ''
            
        # Look for age of acquisition
        df_aoa = df_lang[df_lang.Norm.str.match('^aoa$|^aoa.*_M.*', case=False)]
        if len(df_aoa) > 0:
            df_aoa.sort_values(by=['Score',"Dimensions","Window"], ascending=[False, True, True], inplace=True)
            dim = df_aoa['Dimensions'].iloc[0]
            win = df_aoa['Window'].iloc[0]
            row['Age of Acquisition'] = f'{dim}-{win}'
        else:
            row['Age of Acquisition'] = ''
            
        # Look for valence
        df_val = df_lang[df_lang.Norm.str.match('^valence$|^valence.*_M.*', case=False)]
        if len(df_val) > 0:
            if lang == 'af': print(df_val)
            df_val.sort_values(by=['Score',"Dimensions","Window"], ascending=[False, True, True], inplace=True)
            dim = df_val['Dimensions'].iloc[0]
            win = df_val['Window'].iloc[0]
            row['Valence'] = f'{dim}-{win}'
        else:
            row['Valence'] = ''

        
    
    rows.append(row)
    
final_df = pd.DataFrame(rows)
final_df.sort_values(by=['Language'], ascending=[True], inplace=True)
final_df.columns = ['code', 'Frequency: Wikipedia','Frequency: Subtitles','Imagineability','Age of Acquisition','Valence']

code2lang = os.path.join(basedir, "geo", "code2lang.csv")
code_df = pd.read_csv(code2lang)

final_df = final_df.merge(code_df, on="code")
final_df = final_df.drop(columns=['code'])
final_df = final_df[['Language', 'Frequency: Wikipedia','Frequency: Subtitles','Imagineability','Age of Acquisition','Valence']]
file = os.path.join(basedir,"final_answer.csv")
final_df.to_csv(file)
    

In [40]:
with pd.option_context('display.max_rows', 60):
    display(final_df)

,Language,Frequency: Wikipedia,Frequency: Subtitles,Imagineability,Age of Acquisition,Valence
0,Afrikaans,50-3,300-6,,,
1,Arabic,50-5,50-5,200-1,200-1,
2,Bulgarian,50-1,100-2,,,
3,Bengali,50-3,100-1,,,
4,Breton,50-5,50-2,,,
5,Bosnian,200-1,300-1,,,
6,Catalan,200-1,100-1,,,
7,Czech,50-1,50-1,,,
8,Danish,500-1,500-1,,,
9,German,500-1,500-1,200-1,200-1,500-1
